In [1]:
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import healpy as hp
from astropy.coordinates import SkyCoord, search_around_sky, Angle
from astropy.cosmology import Planck18
import astropy.units as u
import numpy as np

## completeness data
data1 = np.loadtxt('bright_completeness.csv', skiprows = 3, delimiter = ',')
nside = 128

## number of each galaxy and completeness associated with each
healpix_number = data1[:,0]
completeness = data1[:,1]

npix = hp.nside2npix(nside) ## number of pixels

## opening the file
hdul = fits.open("BGS_BRIGHT_NGC_clustering.dat.fits", memmap = True) ## use memmap as the file is big 
hdul.info()

## stuff in the file
data2 = hdul[1].data 

data_table = Table(data2)
hdul.close()

Filename: BGS_BRIGHT_NGC_clustering.dat.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   ()      
  1  LSS           1 BinTableHDU     55   2909876R x 18C   [K, D, K, D, D, 1A, D, D, D, D, D, E, E, E, E, E, D, D]   


In [2]:
## need to get the ra and dec of each galaxy 
ra = data_table['RA']
dec = data_table['DEC']

ra_rad = np.deg2rad(ra) ## convert to radians
dec_rad = np.deg2rad(dec)

theta = 0.5 * np.pi - dec_rad  ## healpix uses theta and phi, not ra and dec
phi = ra_rad

healpix_index = hp.ang2pix(nside, theta, phi) ## generating healpix indices for each galaxy

## making a completeness map array where you index at the healpix number of a galaxy and then it returns the completeness of said galaxy
comp_map = np.zeros(npix) ## making an array of just zeroes the size of the healpix map
comp_map[healpix_number.astype(int)] = completeness ## filling map with the completenesses

comp_cut = comp_map[healpix_index] >= 0.8 ## completenesses of each galaxy in the fits file must be at least 0.8

complete_data = data_table[comp_cut] ## now table only includes samples that are at least 80% complete

In [3]:
## now use new filtered data and use astropy
mpc_rad = 1 * u.Mpc
#part_table = complete_data[1:25000]
z = complete_data['Z']

# coordinates of each galaxy using ra and dec
ra = complete_data['RA']
dec = complete_data['DEC']
coords2 = SkyCoord(ra=ra*u.deg, dec=dec*u.deg) ## coordinates of each galaxy in the data set
d_a = Planck18.angular_diameter_distance(z) ## angular diameter distance

theta = Angle((mpc_rad / d_a).value, unit=u.rad) ## defining the angular radius of the cone

coords1 = []
all_cand = []
all_neigh = []
all_sep = []
chunk = 25000

for start in range(0, len(coords2), chunk):
    end = start + chunk
    coords1 = coords2[start:end]
    cand, neigh, sep, _ = search_around_sky(coords1, coords2, seplimit = theta[start:end], storekdtree = True) ## searching for pairs of galaxies within a specific separation limit

    cand += start
    
    all_cand.append(cand)
    all_neigh.append(neigh)
    all_sep.append(sep)

cand = np.concatenate(all_cand)
neigh = np.concatenate(all_neigh)
sep = np.concatenate(all_sep)

theta_cut = theta[cand] > sep.to(u.rad)

## to make a cut on the redshifts (neighbours must be +/- 0.01 the redshift of the candidates)
delta_z = np.abs(z[cand]-z[neigh]) <= 0.01

In [4]:
## determining luminosity of each galaxy
flux_rbandnmgy = complete_data['flux_r_dered']
app_mags = (22.5 - 2.5 * np.log10(flux_rbandnmgy.value)) * u.mag
lum_dists = Planck18.luminosity_distance(z)
abs_mags = app_mags.value - (5*np.log10(lum_dists.value/0.00001))
magsun_r = 4.64
luminosities = 10**(-0.4 * (abs_mags - magsun_r))

neigh_lum = luminosities[neigh]
cand_lum = luminosities[cand]

lum_cut = neigh_lum/cand_lum > 0.1 ## keep neighbours that are more than 0.1 luminosity of the candidate 

mag_cut = abs_mags[cand] < -17 ## keep candidate galaxies that have a magnitude less than -17

In [5]:
self_cut = cand != neigh
unique_cut = cand > neigh

cut = theta_cut & delta_z & lum_cut & mag_cut & self_cut & unique_cut
## cutting a cone around each independent candidate galaxy and ensures that the cone is larger than separation b/w candidate and neighbour

cand, neigh, sep = cand[cut], neigh[cut], sep[cut] ## applying the cuts

print(f'Candidate galaxies: {cand}')
print(f'Neighbouring galaxies: {neigh}')

Candidate galaxies: [      2       3       3 ... 1481271 1481272 1481272]
Neighbouring galaxies: [      1       1       2 ... 1481268 1481268 1481271]
